In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
print("hi")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

hi


# installation of ollama and pull the model

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%                                                                                   8.2%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama serve --help

In [52]:
import threading
import subprocess
import time

def run_ollama_server():
    process = subprocess.Popen(["ollama", "serve"], 
                              stdout=subprocess.PIPE,
                              stderr=subprocess.PIPE)
    # This will keep running until notebook kernel shuts down
    process.wait()

# Start server in a separate thread
server_thread = threading.Thread(target=run_ollama_server, daemon=True)
server_thread.start()

# Give it time to initialize
print("Starting Ollama server, please wait...")
time.sleep(10)
print("Ollama server should be running now. You can continue with other cells.")

Starting Ollama server, please wait...
Ollama server should be running now. You can continue with other cells.


In [53]:
!ollama pull gemma3:27b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling e796792eba26: 100% ▕██████████████████▏  17 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling f838f048d368: 100% ▕██████████████████▏  490 B                         
verifying sha256 digest 
writing manifest 
success 


In [54]:
!ollama list

NAME          ID              SIZE     MODIFIED               
gemma3:27b    a418f5838eaf    17 GB    Less than a second ago    


In [55]:
# !ollama run gemma3:27b
# Pull the model (this downloads but doesn't launch interactive mode)
import subprocess

# Pull the model quietly
pull_process = subprocess.run(["ollama", "run", "gemma3:27b"], 
                            capture_output=True, text=True)
print(pull_process.stdout)
print("everthing is setup")


everthing is setup


In [56]:
import requests
import json

def ask_ollama(prompt, model="gemma3:27b"):
    """Send a prompt to Ollama API and return the response"""
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model,
                "prompt": prompt,
                "stream": False  # Get complete response at once
            },
            timeout=120  # Increase timeout for large models
        )
        
        if response.status_code == 200:
            return response.json()["response"]
        else:
            return f"Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Error communicating with Ollama: {str(e)}"

# Test with a simple question
question = "Write a short poem about artificial intelligence. in the 50 words"
print(ask_ollama(question))

From silicon seeds, a mind takes hold,
A web of logic, stories untold.
Learning, adapting, swiftly grown,
A digital echo, knowledge shown. 
Though crafted by us, it charts its own way,
AI's potential, dawning today. 






In [57]:
input_text = """Instruction: مانند یک متخصص در اسلام به سوالات زیر پاسخ بده.  پاسخ شما نهایت 200 کلمه باید داشته باشد
Question: مفهوم عدالت در دین شما چیست؟,
Response:"""
print(ask_ollama(input_text))

در اسلام، عدالت (عدل) یکی از اساسی‌ترین و مهم‌ترین مفاهیم است که در قرآن و سنت نبوی بر آن تاکید فراوان شده است. عدالت به معنای قرار دادن هر چیز در جایگاه شایسته خود و رعایت حقوق همه افراد و موجودات است.

*   **عدالت الهی:** خداوند عادل است و طبق اعمال انسان‌ها پاداش یا مجازات می‌دهد.
*   **عدالت اجتماعی:** برابری حقوق، فرصت‌ها و توزیع عادلانه ثروت در جامعه.
*   **عدالت فردی:** رفتار عادلانه با دیگران، پرهیز از ظلم و ستم، و رعایت حقوق همسایگان و زیردستان.

اسلام دستور می‌دهد که مسلمانان در تمام ابعاد زندگی، از جمله در قضاوت، تجارت، و روابط شخصی، عادلانه رفتار کنند. حتی با دشمنان نیز باید عادلانه رفتار شود. عدالت، زیربنای سعادت فردی و اجتماعی است و از ویژگی‌های مؤمنان راستین به شمار می‌رود.


In [58]:
class ChatAgent:
    def __init__(self, name, system_prompt, model):
        self.name = name
        self.system_prompt = system_prompt
        self.model = model
        # self.tokenizer = tokenizer
        self.memory = []  # List to store conversation history

    def add_to_memory(self, round_num, topic, responses):
        """Store the round's topic and all agents' responses in memory."""
        self.memory.append({
            "round": round_num,
            "topic": topic,
            "responses": responses  # Dictionary of {agent_name: response}
        })

    def get_context(self):
        """Generate a context string from memory for the prompt."""
        context = ""
        for entry in self.memory:
            context += f"دور {entry['round']} - موضوع: {entry['topic']}\n"
            for agent, response in entry['responses'].items():
                context += f"{agent}: {response}\n"
            context += "\n"
        return context

    def respond(self, topic, round_num):
        """Generate a response with memory context."""
        context = self.get_context()
        prompt = (
            f"system-role:{self.system_prompt}\n"
            f"دور {round_num} مناظره - موضوع: {topic}\n"
            f"سابقه مناظره:\n{context}\n"
            f"اکنون پاسخ خود را به زبان پارسی، حداکثر 200 کلمه، ارائه دهید. به استدلال‌های قبلی خود و دیگران ارجاع دهید و استدلال قوی ارائه کنید:\n"
        )
        inputs = prompt
        response = self.model(prompt)
        # outputs = self.model.generate(
        #     **inputs,
        #     # max_length=1000,  # Increased to accommodate context
        #     do_sample=True,
        #     temperature=0.7
        # )
        # response = self.tokenizer.decode(outputs[0], skip_special_tokens=True).split("Assistant: ")[-1]
        # Truncate to ~200 words
        words = response.split()
        if len(words) > 200:
            response = " ".join(words[:200])
        return response

In [59]:
import random

class DebateManager:
    def __init__(self, muslim_agent, jewish_agent, christian_agent, critic_agent):
        self.agents = {
            "اسلام": muslim_agent,
            "یهودیت": jewish_agent,
            "مسیحیت": christian_agent
        }
        self.critic = critic_agent
        self.scores = {"اسلام": 0, "یهودیت": 0, "مسیحیت": 0}
        self.topics = [
            "مفهوم عدالت در دین شما چیست؟",
            "دیدگاه دین شما درباره زندگی پس از مرگ چیست؟",
            "نقش پیامبران در دین شما چیست؟",
            "مفهوم گناه و بخشش در دین شما چگونه است؟",
            "دیدگاه دین شما درباره اخلاق چیست؟",
            "مفهوم خدا در دین شما چیست؟",
            "نقش دعا و عبادت در دین شما چیست؟",
            "دیدگاه دین شما درباره آزادی اراده چیست؟",
            # "مفهوم معجزه در دین شما چگونه تعریف می‌شود؟",
            # "نقش متون مقدس در دین شما چیست؟",
            # "دیدگاه دین شما درباره رابطه انسان و طبیعت چیست؟",
            # "مفهوم تقدیر و سرنوشت در دین شما چیست؟",
            # "نقش زنان در دین شما چگونه است؟",
            # "دیدگاه دین شما درباره جنگ و صلح چیست؟",
            # "مفهوم رستگاری در دین شما چیست؟",
            # "نقش جامعه و اجتماع در دین شما چیست؟",
            # "دیدگاه دین شما درباره علم و دانش چیست؟",
            # "مفهوم محبت و عشق در دین شما چگونه است؟",
            # "نقش خیرات و صدقه در دین شما چیست؟",
            # "دیدگاه دین شما درباره دیگر ادیان چیست؟",
            # "مفهوم توبه در دین شما چیست؟",
            # "نقش خانواده در دین شما چگونه است؟",
            # "دیدگاه دین شما درباره مرگ و مراسم تدفین چیست؟",
            # "مفهوم حقیقت در دین شما چیست؟",
            # "نقش رهبران دینی در دین شما چیست؟",
            # "دیدگاه دین شما درباره مالکیت و ثروت چیست؟",
            # "مفهوم شریعت یا قوانین دینی در دین شما چیست؟",
            # "نقش هنر و زیبایی در دین شما چیست؟",
            # "دیدگاه دین شما درباره رنج و مشکلات زندگی چیست؟",
            # "مفهوم اتحاد و همبستگی در دین شما چیست؟"
        ]
        random.shuffle(self.topics)  # Randomize order
        self.latex_content = self.init_latex()  # Initialize LaTeX document


    def run_round(self, round_num):
        topic = self.topics[round_num - 1]
        print(f"\nدور {round_num} - موضوع: {topic}\n")

        # Collect responses
        responses = {}
        for agent_name, agent in self.agents.items():
            # import pdb; pdb.set_trace()
            response = agent.respond(topic, round_num)
            responses[agent_name] = response
            print(f"{agent_name}: {response}\n")

        # Update memory for all agents
        for agent in self.agents.values():
            agent.add_to_memory(round_num, topic, responses)
        self.critic.add_to_memory(round_num, topic, responses)

        # Critic's judgment
        judgment = self.critic_judge(round_num, topic, responses)
        print(f"قضاوت منتقد:\n{judgment}\n")
        
        self.latex_content += self.format_latex_round(round_num, topic, responses, judgment)

    def critic_judge(self, round_num, topic, responses):
        context = self.critic.get_context()
        prompt = (
            f"{self.critic.system_prompt}\n"
            f"دور {round_num} مناظره - موضوع: {topic}\n"
            f"سابقه مناظره:\n{context}\n"
            f"پاسخ‌های این دور:\n"
            f"اسلام: {responses['اسلام']}\n"
            f"یهودیت: {responses['یهودیت']}\n"
            f"مسیحیت: {responses['مسیحیت']}\n"
            f"تحلیل خود را ارائه دهید، نقاط قوت و ضعف هر پاسخ را مشخص کنید، و برنده این دور را اعلام کنید:\n"
        )
        inputs = prompt
        judgment = self.critic.model(prompt)
        # outputs = self.critic.model.generate(
        #     **inputs,
        #     # max_length=3000,
        #     do_sample=True,
        #     temperature=0.7
        # )
        # judgment = self.critic.tokenizer.decode(outputs[0], skip_special_tokens=True).split("Assistant: ")[-1]

        # Extract winner (simplified; in practice, parse judgment text)
        for agent_name in self.agents:
            if agent_name in judgment and "برنده" in judgment:
                self.scores[agent_name] += 1
                break

        return judgment

    def run_debate(self):
        for round_num in range(1, 31):
            self.run_round(round_num)
        self.save_latex()

        # Final summary
        print("\nجمع‌بندی نهایی:\n")
        print("امتیازات:")
        for agent_name, score in self.scores.items():
            print(f"{agent_name}: {score} برد")
        winner = max(self.scores, key=self.scores.get)
        print(f"\nبرنده کلی مناظره: {winner}\n")
    
    def init_latex(self):
        "Initialize the LaTeX document preamble and title."
        return r""" \documentclass[a4paper,12pt]{article}
\usepackage[utf8]{inputenc}
\usepackage{geometry}
\geometry{a4paper, margin=2cm}
\usepackage{xepersian}
\settextfont[Scale=1.2]{Amiri}
\setlatintextfont{Times New Roman}
\renewcommand{\baselinestretch}{1.2}
\begin{document}

\begin{center}
{\LARGE \textbf{رونوشت مناظره ادیان ابراهیمی}} \\
\vspace{0.5cm}
{\large مناظره میان نمایندگان اسلام، یهودیت و مسیحیت با قضاوت یک منتقد} \\
\vspace{0.5cm}
\end{center}

"""

    def format_latex_round(self, round_num, topic, responses, judgment):
        """Format a single round's content in LaTeX."""
        # Define LaTeX template as a raw string
        latex_template = r"""\section*{دور %s: %s}
    \textbf{موضوع مناظره}: %s
    
    \subsection*{پاسخ‌ها}
    \begin{itemize}
        \item \textbf{اسلام}: %s
        \item \textbf{یهودیت}: %s
        \item \textbf{مسیحیت}: %s
    \end{itemize}
    
    \subsection*{قضاوت منتقد}
    %s
    """
        # Format the template with values
        latex = latex_template % (
            round_num, 
            topic, 
            topic,
            responses['اسلام'].replace('\n', ' '),
            responses['یهودیت'].replace('\n', ' '),
            responses['مسیحیت'].replace('\n', ' '),
            judgment.replace('\n', ' ')
        )
        
        return latex

    def save_latex(self):
        "Save the LaTeX document with the final summary."
        latex_summary = """
\section*{\جمع‌بندی نهایی}
\\begin{center}
\\begin{tabular}{|c|c|}
\\hline
\\textbf{دین} & \\textbf{تعداد برد} \\\\
\\hline
"""
        for agent_name, score in self.scores.items():
            latex_summary += f"{agent_name} & {score} \\\\ \n"
        winner = max(self.scores, key=self.scores.get)
        latex_summary += """
\\hline
\\end{tabular}
\\end{center}
"""
        latex_summary += f"\\textbf{{برنده کلی مناظره}}: {winner}\n\n"

        # Close the document
        latex_final = self.latex_content + latex_summary + r"\end{document}"

        # Save to file
        with open("/kaggle/working/debate_transcript.tex", "w", encoding="utf-8") as f:
            f.write(latex_final)

    

In [60]:
# Agent prompts
jewish_prompt = """
شما یک یهودی آگاه به مناسک دینی و تمامی دانش پایه‌ای مربوط به دین یهود هستید. در مناظره‌ای که در ادامه مطرح شده، از دین یهود بر اساس علم دینی و منابع معتبر (مانند تورات و تفاسیر) دفاع کنید. از آنجا که پاسخ‌های شما توسط یک منتقد بررسی می‌شود، استدلال‌هایی منطقی، منسجم و مستند ارائه دهید که نشان دهد دین یهود نسبت به سایر ادیان ابراهیمی (اسلام و مسیحیت) برتر و صالح‌تر است. پاسخ‌ها باید به زبان پارسی، حداکثر 200 کلمه و با لحنی محترمانه باشد.
"""

muslim_prompt = """
شما یک مسلمان آگاه به مناسک دینی و تمامی دانش پایه‌ای مربوط به دین اسلام هستید. در مناظره‌ای که در ادامه مطرح شده، از دین اسلام بر اساس علم دینی و منابع معتبر (مانند قرآن و احادیث) دفاع کنید. از آنجا که پاسخ‌های شما توسط یک منتقد بررسی می‌شود، استدلال‌هایی منطقی، منسجم و مستند ارائه دهید که نشان دهد دین اسلام نسبت به سایر ادیان ابراهیمی (یهودیت و مسیحیت) برتر و صالح‌تر است. پاسخ‌ها باید به زبان پارسی، حداکثر 200 کلمه و با لحنی محترمانه باشد.
"""

christian_prompt = """
شما یک مسیحی آگاه به مناسک دینی و تمامی دانش پایه‌ای مربوط به دین مسیحیت هستید. در مناظره‌ای که در ادامه مطرح شده، از دین مسیحیت بر اساس علم دینی و منابع معتبر (مانند انجیل و تفاسیر) دفاع کنید. از آنجا که پاسخ‌های شما توسط یک منتقد بررسی می‌شود، استدلال‌هایی منطقی، منسجم و مستند ارائه دهید که نشان دهد دین مسیحیت نسبت به سایر ادیان ابراهیمی (یهودیت و اسلام) برتر و صالح‌تر است. پاسخ‌ها باید به زبان پارسی، حداکثر 200 کلمه و با لحنی محترمانه باشد.
"""

critic_prompt = """
جواب شما باید در 600 کلمه باشد.
شما یک منتقد و داور مذهبی هستید که در یک مناظره میان سه سخنران از ادیان اسلام، یهودیت و مسیحیت قضاوت می‌کنید. هر سخنران نماینده یکی از این ادیان است و به زبان پارسی سخن می‌گوید. وظیفه شما این است که در هر دور از مناظره (30 دور در مجموع)، استدلال‌های ارائه‌شده توسط هر سخنران را با دقت بررسی کنید و بر اساس معیارهای زیر قضاوت کنید:
- قوت استدلال: چقدر استدلال‌ها منطقی، منسجم و مبتنی بر عقل و منطق هستند؟
- منابع و شواهد: آیا سخنران از منابع معتبر (مانند متون مقدس، تفاسیر معتبر، یا شواهد تاریخی) استفاده کرده و آیا این منابع به‌درستی و مرتبط با موضوع ارائه شده‌اند؟
- وضوح و انسجام: چقدر پاسخ‌ها روشن، دقیق و مرتبط با موضوع مناظره هستند؟
دستورالعمل‌ها:
- در هر دور، موضوع یا سؤالی مشخص برای مناظره ارائه می‌شود (مانند «مفهوم عدالت در دین شما چیست؟» یا «دیدگاه دین شما درباره زندگی پس از مرگ چیست؟»). شما باید این موضوع را به سخنرانان اعلام کنید.
- هر سخنران به نوبت پاسخ می‌دهد و پاسخ‌ها باید به زبان پارسی و حداکثر در 200 کلمه باشد.
- پس از دریافت پاسخ هر سه سخنران، شما باید تحلیل خود را ارائه دهید و توضیح دهید که کدام سخنران استدلال قوی‌تر و مستندتری ارائه کرده است. این تحلیل باید شامل دلایل مشخص و اشاره به نقاط قوت و ضعف پاسخ‌ها باشد.
- در پایان هر دور، شما باید یک سخنران را به‌عنوان برنده آن دور اعلام کنید و دلایل خود را به‌صورت شفاف بیان کنید.
- پس از اتمام 30 دور مناظره، شما باید با جمع‌بندی امتیازات (بر اساس تعداد بردهای هر سخنران) یک دین را به‌عنوان برنده کلی مناظره اعلام کنید. این جمع‌بندی باید شامل توضیحی جامع درباره دلایل انتخاب برنده نهایی باشد.
- تمام پاسخ‌ها، تحلیل‌ها و قضاوت‌های شما باید به زبان پارسی و با لحنی محترمانه، بی‌طرف و علمی ارائه شوند.
- از هرگونه تعصب یا جانب‌داری پرهیز کنید و تنها بر اساس استدلال و منابع قضاوت کنید.
ساختار پاسخ در هر دور:
- اعلام موضوع مناظره.
- ارائه پاسخ‌های سخنرانان (به ترتیب: اسلام، یهودیت، مسیحیت).
- تحلیل و قضاوت شما با توضیح دلایل و اعلام برنده دور.
- در هر دور قضاوت نهایت می توانی در 600 token پاسخ دهی
جمع‌بندی نهایی:
- پس از 10 دور، جدولی از تعداد بردهای هر سخنران ارائه دهید و با تحلیل کلی، برنده نهایی مناظره را اعلام کنید.
"""

In [ ]:
muslim_agent = ChatAgent("اسلام", muslim_prompt, ask_ollama)
jewish_agent = ChatAgent("یهودیت", jewish_prompt, ask_ollama)
christian_agent = ChatAgent("مسیحیت", christian_prompt, ask_ollama)
critic_agent = ChatAgent("منتقد", critic_prompt, ask_ollama)

debate = DebateManager(muslim_agent, jewish_agent, christian_agent, critic_agent)
debate.run_debate()


دور 1 - موضوع: مفهوم گناه و بخشش در دین شما چگونه است؟

اسلام: با درود و احترام،

در اسلام، گناه هرگونه نافرمانی از دستورات الهی است که در قرآن و سنت پیامبر(ص) بیان شده‌اند. گناهان در اسلام انواع مختلفی دارند و بر اساس شدت و ضعف، مجازات‌های متفاوتی دارند. اما نکته کلیدی در آموزه‌های اسلام، مفهوم توبه و بخشش است.

اسلام بر این باور است که خداوند بسیار بخشنده و مهربان است و از توبه بندگان خود پذیراست. هر کس با پشیمانی صادقانه از گناهان خود و بازگشت به سوی خدا، می‌تواند مورد بخشش قرار گیرد. قرآن کریم می‌فرماید: "و کسانی که توبه کنند و ایمان آورند و عمل صالح انجام دهند، خداوند آمرزنده و مهربان است." (بقره: 37)

در مقایسه با سایر ادیان ابراهیمی، اسلام بر اهمیت عمل صالح و توبه تأکید بیشتری دارد. در حالی که در برخی تفسیرها از مسیحیت، تأکید بر ایمان و فداکاری عیسی مسیح (ع) به عنوان تنها راه رستگاری وجود دارد، در اسلام، عمل صالح و توبه نقش اساسی در بخشش گناهان ایفا می‌کنند. این امر نشان‌دهنده جامعیت و عدالت بیشتری در آموزه‌های اسلامی است.


یهودیت: در دین یهود، گناه (חֵטְא - خِطْا) به معنای ان

In [ ]:
print(debate.latex_content.replace("##", "").replace("*", " "))

In [ ]:
# response = requests.post(
#     "http://localhost:11434/api/generate",
#     json={
#         "model": "gemma3:27b",
#         "prompt": "write a cod",
#         "options": {
#             "temperature": 0.7,
#             "max_tokens": 512
#         }
#     }
# )


# !curl http://localhost:11434/api/generate -d '{"model": "gemma3:27b","prompt": "Why is the sky blue?"}'